In [5]:
%reset -fs

In [64]:
import pandas as pd
import numpy as np
import time, os
from bs4 import BeautifulSoup
import datetime
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from pymongo import MongoClient
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

In [7]:
base = datetime.datetime.today()
delta = base - datetime.datetime(2021, 10, 19)
date_list = [base - datetime.timedelta(days=x) for x in range(delta.days+1)]
date_list = [d.strftime('%Y-%m-%d') for d in date_list]

In [8]:
#run this initially; then don't run again if you just want to keep adding to it
final_df = pd.DataFrame()

In [9]:
bet_search =  'https://www.bettingpros.com/nba/picks/prop-bets/bet/points/?date='

In [11]:
chromedriver = "/Applications/chromedriver"
os.environ['webdriver.chrome.driver'] = chromedriver

In [12]:
driver = webdriver.Chrome(chromedriver)

In [13]:
# Tip of the hat to Rita for helping with these two functions !
def get_site_rb(date):
    bet_query = bet_search + date
    driver.get(bet_query)
    time.sleep(5)
    try:
        element = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
        action = ActionChains(driver)
        action.move_to_element(element).click().perform()
    except:
        pass
    get_data_rb()


In [14]:
def get_data_rb():
    data_str = []
    data = []
    li = []
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table')
    date = soup.find('h1').text
    date = date.split(" ")[5]
    date.strip(')')
    date = date.replace('(',"").replace(')',"")
    house_accuracy = soup.find('div', class_='best-props__accuracy').text
    table_data = table.tbody.find_all('tr')
    for tr in table_data:
        data_str.append(tr.text.replace('\n', ' ').strip())
    for i in data_str:
        li = list(i.split('            '))
        data.append(li)
    for i in data:
        if len(i) == 7:
            i.append('TBD')
            i.append(date)
            i.append(house_accuracy)
        else:
            i.append(date)
            i.append(house_accuracy)
    global final_df
    final_df = pd.concat([final_df, pd.DataFrame(data)])
    time.sleep(1)

    try:
        page_progress = driver.find_element_by_xpath('//div[contains(@class, "table__pagination")]').text.strip("«").strip("»").strip()
        if int(page_progress[5]) == int(page_progress[10]):
            return final_df
        else:
            element = driver.find_element_by_xpath('/html/body/div[2]/main/div/div/div/div[4]/div/div/a[2]')
            action = ActionChains(driver)
            action.move_to_element(element).click().perform()
            time.sleep(3)
            get_data_rb()
    except:
        return final_df


In [15]:
#run this and then close the cookies pop-up!
for i in date_list:
    
    if i == '2021-12-24':
        continue
    if i == '2021-11-25':
        continue
    if i == '2022-02-18':
        continue
    if i == '2022-02-19':
        continue
    if i == '2022-02-20':
        continue
    if i == '2022-02-21':
        continue
    if i == '2022-02-22':
        continue
    if i == '2022-02-23':
        continue
    get_site_rb(i)

In [16]:
final_df

,0,1,2,3,4,5,6,7,8,9
0,1 K. Kevin Love,CLE - F,@ ATL,Points,12.50 (-115)19.30Diff: 6.80 Line: 12.50...,12.5,-115 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
1,2 B. Bobby Portis Jr.,MIL - F,@ BKN,Points,7.50 (-125)13.10Diff: 5.60 Line: 7.50 C...,7.5,-125 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
2,3 N. Nikola Vucevic,CHI - C,vs. LAC,Points,19.50 (-120)15.00Diff: -4.50 Line: 19.5...,19.5,-120 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
3,4 G. Goran Dragic,BKN - G,vs. MIL,Points,7.50 (-125)12.00Diff: 4.50 Line: 7.50 C...,7.5,-125 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
4,5 P. Paul George,LAC - F,@ CHI,Points,25.50 (-130)21.20Diff: -4.30 Line: 25.5...,25.5,-130 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
...,...,...,...,...,...,...,...,...,...,...
21,22 G. Grayson Allen,MIL - G,vs. BKN,Points,10.50 (-113)11.23Diff: 0.73 Line: 10.50...,10.5,-113,10 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...
22,23 N. Nemanja Bjelica,GSW - F,@ LAL,Points,6.50 (-120)6.87Diff: 0.37 Line: 6.50 Co...,6.5,-120,15 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...
23,24 D. DeAndre Jordan,LAL - C,vs. GSW,Points,4.50 (-130)4.82Diff: 0.32 Line: 4.50 Co...,4.5,-130,2 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...
24,25 P. Pat Connaughton,MIL - G,vs. BKN,Points,7.50 (-111)7.19Diff: -0.31 Line: 7.50 C...,7.5,-111,20 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...


In [17]:
driver.close()

In [18]:
#Clean up below
final_df.columns=['Name', 'Team', 'Opp', 'Type', 'Line', 'Pick', 'Moneyline', 'Result', 'Date', 'Accuracy']

In [19]:
final_df.reset_index(drop=True, inplace=True)

In [22]:
final_df

,Name,Team,Opp,Type,Line,Pick,Moneyline,Result,Date,Accuracy
0,1 K. Kevin Love,CLE - F,@ ATL,Points,12.50 (-115)19.30Diff: 6.80 Line: 12.50...,12.5,-115 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
1,2 B. Bobby Portis Jr.,MIL - F,@ BKN,Points,7.50 (-125)13.10Diff: 5.60 Line: 7.50 C...,7.5,-125 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
2,3 N. Nikola Vucevic,CHI - C,vs. LAC,Points,19.50 (-120)15.00Diff: -4.50 Line: 19.5...,19.5,-120 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
3,4 G. Goran Dragic,BKN - G,vs. MIL,Points,7.50 (-125)12.00Diff: 4.50 Line: 7.50 C...,7.5,-125 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
4,5 P. Paul George,LAC - F,@ CHI,Points,25.50 (-130)21.20Diff: -4.30 Line: 25.5...,25.5,-130 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
...,...,...,...,...,...,...,...,...,...,...
14477,22 G. Grayson Allen,MIL - G,vs. BKN,Points,10.50 (-113)11.23Diff: 0.73 Line: 10.50...,10.5,-113,10 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...
14478,23 N. Nemanja Bjelica,GSW - F,@ LAL,Points,6.50 (-120)6.87Diff: 0.37 Line: 6.50 Co...,6.5,-120,15 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...
14479,24 D. DeAndre Jordan,LAL - C,vs. GSW,Points,4.50 (-130)4.82Diff: 0.32 Line: 4.50 Co...,4.5,-130,2 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...
14480,25 P. Pat Connaughton,MIL - G,vs. BKN,Points,7.50 (-111)7.19Diff: -0.31 Line: 7.50 C...,7.5,-111,20 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...


In [23]:
player_name = final_df.Name.str.split(" ", expand = True)
final_df['Name'] = player_name[2] + ' ' + player_name[4]

In [24]:
final_df

,Name,Team,Opp,Type,Line,Pick,Moneyline,Result,Date,Accuracy
0,Kevin Love,CLE - F,@ ATL,Points,12.50 (-115)19.30Diff: 6.80 Line: 12.50...,12.5,-115 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
1,Bobby Portis,MIL - F,@ BKN,Points,7.50 (-125)13.10Diff: 5.60 Line: 7.50 C...,7.5,-125 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
2,Nikola Vucevic,CHI - C,vs. LAC,Points,19.50 (-120)15.00Diff: -4.50 Line: 19.5...,19.5,-120 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
3,Goran Dragic,BKN - G,vs. MIL,Points,7.50 (-125)12.00Diff: 4.50 Line: 7.50 C...,7.5,-125 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
4,Paul George,LAC - F,@ CHI,Points,25.50 (-130)21.20Diff: -4.30 Line: 25.5...,25.5,-130 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
...,...,...,...,...,...,...,...,...,...,...
14477,Grayson Allen,MIL - G,vs. BKN,Points,10.50 (-113)11.23Diff: 0.73 Line: 10.50...,10.5,-113,10 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...
14478,Nemanja Bjelica,GSW - F,@ LAL,Points,6.50 (-120)6.87Diff: 0.37 Line: 6.50 Co...,6.5,-120,15 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...
14479,DeAndre Jordan,LAL - C,vs. GSW,Points,4.50 (-130)4.82Diff: 0.32 Line: 4.50 Co...,4.5,-130,2 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...
14480,Pat Connaughton,MIL - G,vs. BKN,Points,7.50 (-111)7.19Diff: -0.31 Line: 7.50 C...,7.5,-111,20 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...


In [25]:
final_df.reset_index(inplace=True, drop=True)

In [26]:
final_df

,Name,Team,Opp,Type,Line,Pick,Moneyline,Result,Date,Accuracy
0,Kevin Love,CLE - F,@ ATL,Points,12.50 (-115)19.30Diff: 6.80 Line: 12.50...,12.5,-115 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
1,Bobby Portis,MIL - F,@ BKN,Points,7.50 (-125)13.10Diff: 5.60 Line: 7.50 C...,7.5,-125 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
2,Nikola Vucevic,CHI - C,vs. LAC,Points,19.50 (-120)15.00Diff: -4.50 Line: 19.5...,19.5,-120 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
3,Goran Dragic,BKN - G,vs. MIL,Points,7.50 (-125)12.00Diff: 4.50 Line: 7.50 C...,7.5,-125 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
4,Paul George,LAC - F,@ CHI,Points,25.50 (-130)21.20Diff: -4.30 Line: 25.5...,25.5,-130 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
...,...,...,...,...,...,...,...,...,...,...
14477,Grayson Allen,MIL - G,vs. BKN,Points,10.50 (-113)11.23Diff: 0.73 Line: 10.50...,10.5,-113,10 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...
14478,Nemanja Bjelica,GSW - F,@ LAL,Points,6.50 (-120)6.87Diff: 0.37 Line: 6.50 Co...,6.5,-120,15 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...
14479,DeAndre Jordan,LAL - C,vs. GSW,Points,4.50 (-130)4.82Diff: 0.32 Line: 4.50 Co...,4.5,-130,2 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...
14480,Pat Connaughton,MIL - G,vs. BKN,Points,7.50 (-111)7.19Diff: -0.31 Line: 7.50 C...,7.5,-111,20 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...


In [28]:
team = final_df.Team.str.split(" ", expand = True)
final_df['Team'] = team[0]
final_df['Position'] = team[2]

In [29]:
final_df

,Name,Team,Opp,Type,Line,Pick,Moneyline,Result,Date,Accuracy,Position
0,Kevin Love,CLE,@ ATL,Points,12.50 (-115)19.30Diff: 6.80 Line: 12.50...,12.5,-115 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...,F
1,Bobby Portis,MIL,@ BKN,Points,7.50 (-125)13.10Diff: 5.60 Line: 7.50 C...,7.5,-125 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...,F
2,Nikola Vucevic,CHI,vs. LAC,Points,19.50 (-120)15.00Diff: -4.50 Line: 19.5...,19.5,-120 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...,C
3,Goran Dragic,BKN,vs. MIL,Points,7.50 (-125)12.00Diff: 4.50 Line: 7.50 C...,7.5,-125 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...,G
4,Paul George,LAC,@ CHI,Points,25.50 (-130)21.20Diff: -4.30 Line: 25.5...,25.5,-130 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...,F
...,...,...,...,...,...,...,...,...,...,...,...
14477,Grayson Allen,MIL,vs. BKN,Points,10.50 (-113)11.23Diff: 0.73 Line: 10.50...,10.5,-113,10 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...,G
14478,Nemanja Bjelica,GSW,@ LAL,Points,6.50 (-120)6.87Diff: 0.37 Line: 6.50 Co...,6.5,-120,15 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...,F
14479,DeAndre Jordan,LAL,vs. GSW,Points,4.50 (-130)4.82Diff: 0.32 Line: 4.50 Co...,4.5,-130,2 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...,C
14480,Pat Connaughton,MIL,vs. BKN,Points,7.50 (-111)7.19Diff: -0.31 Line: 7.50 C...,7.5,-111,20 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...,G


In [30]:
opp = final_df.Opp.str.split(" ", expand = True)
final_df.Opp = opp[1]
final_df['Home/Away'] = opp[0]

In [31]:
final_df

,Name,Team,Opp,Type,Line,Pick,Moneyline,Result,Date,Accuracy,Position,Home/Away
0,Kevin Love,CLE,ATL,Points,12.50 (-115)19.30Diff: 6.80 Line: 12.50...,12.5,-115 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...,F,@
1,Bobby Portis,MIL,BKN,Points,7.50 (-125)13.10Diff: 5.60 Line: 7.50 C...,7.5,-125 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...,F,@
2,Nikola Vucevic,CHI,LAC,Points,19.50 (-120)15.00Diff: -4.50 Line: 19.5...,19.5,-120 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...,C,vs.
3,Goran Dragic,BKN,MIL,Points,7.50 (-125)12.00Diff: 4.50 Line: 7.50 C...,7.5,-125 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...,G,vs.
4,Paul George,LAC,CHI,Points,25.50 (-130)21.20Diff: -4.30 Line: 25.5...,25.5,-130 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...,F,@
...,...,...,...,...,...,...,...,...,...,...,...,...
14477,Grayson Allen,MIL,BKN,Points,10.50 (-113)11.23Diff: 0.73 Line: 10.50...,10.5,-113,10 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...,G,vs.
14478,Nemanja Bjelica,GSW,LAL,Points,6.50 (-120)6.87Diff: 0.37 Line: 6.50 Co...,6.5,-120,15 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...,F,@
14479,DeAndre Jordan,LAL,GSW,Points,4.50 (-130)4.82Diff: 0.32 Line: 4.50 Co...,4.5,-130,2 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...,C,vs.
14480,Pat Connaughton,MIL,BKN,Points,7.50 (-111)7.19Diff: -0.31 Line: 7.50 C...,7.5,-111,20 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...,G,vs.


In [32]:
bet_info = final_df.Line.str.split(" ", expand = True)

In [33]:
final_df['Line'] = bet_info[7]
final_df['Projection'] = bet_info[15]
final_df['Diff'] = bet_info[9]
final_df['Moneyline'] = bet_info[13]
final_df['over_under'] = bet_info[16]
final_df['over_under'] = final_df['over_under'].fillna('o')
final_df

,Name,Team,Opp,Type,Line,Pick,Moneyline,Result,Date,Accuracy,Position,Home/Away,Projection,Diff,over_under
0,Kevin Love,CLE,ATL,Points,12.50,12.5,-115,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...,F,@,19.30o,6.80,o
1,Bobby Portis,MIL,BKN,Points,7.50,7.5,-125,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...,F,@,13.10o,5.60,o
2,Nikola Vucevic,CHI,LAC,Points,19.50,19.5,-120,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...,C,vs.,15.00,-4.50,u
3,Goran Dragic,BKN,MIL,Points,7.50,7.5,-125,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...,G,vs.,12.00o,4.50,o
4,Paul George,LAC,CHI,Points,25.50,25.5,-130,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...,F,@,21.20,-4.30,u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14477,Grayson Allen,MIL,BKN,Points,10.50,10.5,-113,10 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...,G,vs.,11.23o,0.73,o
14478,Nemanja Bjelica,GSW,LAL,Points,6.50,6.5,-120,15 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...,F,@,6.87o,0.37,o
14479,DeAndre Jordan,LAL,GSW,Points,4.50,4.5,-130,2 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...,C,vs.,4.82o,0.32,o
14480,Pat Connaughton,MIL,BKN,Points,7.50,7.5,-111,20 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1145-1109 (5...,G,vs.,7.19,-0.31,u


In [34]:
final_df = final_df[['Date', 'Name', 'Team', 'Position', 'Opp','Home/Away', 'Line', 'Moneyline', 'Projection', 'Diff', 'Pick', 'Result', 'Accuracy', 'over_under']]

In [35]:
final_df = final_df.copy()

In [36]:
final_df['Projection'] = final_df['Projection'].map(lambda x: x.rstrip('ou'))
final_df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,Accuracy,over_under
0,03-31-2022,Kevin Love,CLE,F,ATL,@,12.50,-115,19.30,6.80,12.5,TBD,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...,o
1,03-31-2022,Bobby Portis,MIL,F,BKN,@,7.50,-125,13.10,5.60,7.5,TBD,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...,o
2,03-31-2022,Nikola Vucevic,CHI,C,LAC,vs.,19.50,-120,15.00,-4.50,19.5,TBD,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...,u
3,03-31-2022,Goran Dragic,BKN,G,MIL,vs.,7.50,-125,12.00,4.50,7.5,TBD,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...,o
4,03-31-2022,Paul George,LAC,F,CHI,@,25.50,-130,21.20,-4.30,25.5,TBD,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...,u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14477,10-19-2021,Grayson Allen,MIL,G,BKN,vs.,10.50,-113,11.23,0.73,10.5,10 (under),Season-to-date Top 15 Accuracy: 1145-1109 (5...,o
14478,10-19-2021,Nemanja Bjelica,GSW,F,LAL,@,6.50,-120,6.87,0.37,6.5,15 (over),Season-to-date Top 15 Accuracy: 1145-1109 (5...,o
14479,10-19-2021,DeAndre Jordan,LAL,C,GSW,vs.,4.50,-130,4.82,0.32,4.5,2 (under),Season-to-date Top 15 Accuracy: 1145-1109 (5...,o
14480,10-19-2021,Pat Connaughton,MIL,G,BKN,vs.,7.50,-111,7.19,-0.31,7.5,20 (over),Season-to-date Top 15 Accuracy: 1145-1109 (5...,u


In [37]:
accuracies = final_df.Accuracy.str.split(" ", expand = True)

In [38]:
final_df['seven_day_accuracy'] = accuracies[8]

In [39]:
final_df['seven_day_accuracy'] = final_df['seven_day_accuracy'].map(lambda x: x.lstrip('(').rstrip('%)'))

In [40]:
final_df.seven_day_accuracy = final_df.seven_day_accuracy.replace('', 50, regex=True)

In [41]:
final_df['seven_day_accuracy'] = pd.to_numeric(final_df['seven_day_accuracy'])

In [42]:
final_df['seven_day_accuracy'] = final_df['seven_day_accuracy'].map(lambda x: x/100)

In [43]:
final_df.drop(columns=['Accuracy'], inplace=True)

In [44]:
final_df['Date'] = pd.to_datetime(final_df['Date'])

In [45]:
final_df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,over_under,seven_day_accuracy
0,2022-03-31,Kevin Love,CLE,F,ATL,@,12.50,-115,19.30,6.80,12.5,TBD,o,0.46
1,2022-03-31,Bobby Portis,MIL,F,BKN,@,7.50,-125,13.10,5.60,7.5,TBD,o,0.46
2,2022-03-31,Nikola Vucevic,CHI,C,LAC,vs.,19.50,-120,15.00,-4.50,19.5,TBD,u,0.46
3,2022-03-31,Goran Dragic,BKN,G,MIL,vs.,7.50,-125,12.00,4.50,7.5,TBD,o,0.46
4,2022-03-31,Paul George,LAC,F,CHI,@,25.50,-130,21.20,-4.30,25.5,TBD,u,0.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14477,2021-10-19,Grayson Allen,MIL,G,BKN,vs.,10.50,-113,11.23,0.73,10.5,10 (under),o,0.50
14478,2021-10-19,Nemanja Bjelica,GSW,F,LAL,@,6.50,-120,6.87,0.37,6.5,15 (over),o,0.50
14479,2021-10-19,DeAndre Jordan,LAL,C,GSW,vs.,4.50,-130,4.82,0.32,4.5,2 (under),o,0.50
14480,2021-10-19,Pat Connaughton,MIL,G,BKN,vs.,7.50,-111,7.19,-0.31,7.5,20 (over),u,0.50


In [46]:
# Final form betting df, additional cleaning and saving to csv below
sorted_df = final_df.sort_values(by=['Name', 'Date'])
sorted_df.reset_index(inplace=True, drop=True)

In [47]:
sorted_df['Line'] = pd.to_numeric(sorted_df.Line)

In [48]:
sorted_df = sorted_df.replace('EVEN', '+100')

In [49]:
sorted_df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,over_under,seven_day_accuracy
0,2021-10-21,Bogdan,ATL,G,DAL,vs.,15.5,-115,16.57,1.07,15.5,11 (under),o,0.47
1,2021-10-23,Bogdan,ATL,G,CLE,@,14.5,-115,15.29,0.79,14.5,5 (under),o,0.45
2,2021-10-25,Bogdan,ATL,G,DET,vs.,15.5,-115,12.85,-2.65,15.5,14 (over),u,0.45
3,2021-10-27,Bogdan,ATL,G,NOR,@,12.5,-125,13.70,1.20,12.5,12 (under),o,0.45
4,2021-10-28,Bogdan,ATL,G,WAS,@,12.5,-105,16.72,4.22,12.5,14 (over),o,0.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14477,2022-03-08,Ziaire Williams,MEM,G,NOR,vs.,6.5,-110,7.30,0.80,6.5,16 (over),o,0.46
14478,2022-03-11,Ziaire Williams,MEM,G,NYK,vs.,7.5,-115,7.30,-0.20,7.5,13 (over),u,0.50
14479,2022-03-15,Ziaire Williams,MEM,G,IND,@,8.5,-130,7.60,-0.90,8.5,11 (over),u,0.46
14480,2022-03-23,Ziaire Williams,MEM,G,BKN,vs.,7.5,-105,6.20,-1.30,7.5,0 (under),u,0.46


In [50]:
sorted_df = sorted_df.replace('TBD', None)

In [51]:
sorted_df['Moneyline'] = pd.to_numeric(sorted_df.Moneyline)

In [52]:
sorted_df['Result'] = sorted_df.Result.apply(lambda x: str(x));sorted_df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,over_under,seven_day_accuracy
0,2021-10-21,Bogdan,ATL,G,DAL,vs.,15.5,-115,16.57,1.07,15.5,11 (under),o,0.47
1,2021-10-23,Bogdan,ATL,G,CLE,@,14.5,-115,15.29,0.79,14.5,5 (under),o,0.45
2,2021-10-25,Bogdan,ATL,G,DET,vs.,15.5,-115,12.85,-2.65,15.5,14 (over),u,0.45
3,2021-10-27,Bogdan,ATL,G,NOR,@,12.5,-125,13.70,1.20,12.5,12 (under),o,0.45
4,2021-10-28,Bogdan,ATL,G,WAS,@,12.5,-105,16.72,4.22,12.5,14 (over),o,0.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14477,2022-03-08,Ziaire Williams,MEM,G,NOR,vs.,6.5,-110,7.30,0.80,6.5,16 (over),o,0.46
14478,2022-03-11,Ziaire Williams,MEM,G,NYK,vs.,7.5,-115,7.30,-0.20,7.5,13 (over),u,0.50
14479,2022-03-15,Ziaire Williams,MEM,G,IND,@,8.5,-130,7.60,-0.90,8.5,11 (over),u,0.46
14480,2022-03-23,Ziaire Williams,MEM,G,BKN,vs.,7.5,-105,6.20,-1.30,7.5,0 (under),u,0.46


In [55]:
results = sorted_df.Result.str.split(' ', expand=True);results[3]

0        11
1         5
2        14
3        12
4        14
         ..
14477    16
14478    13
14479    11
14480     0
14481     3
Name: 3, Length: 14482, dtype: object

In [56]:
sorted_df.Result = results[3]

In [60]:
sorted_df['Projection'] = pd.to_numeric(sorted_df.Projection)
sorted_df['Diff'] = pd.to_numeric(sorted_df.Diff)
sorted_df['Result'] = pd.to_numeric(sorted_df.Result)
sorted_df['Pick'] = pd.to_numeric(sorted_df.Pick)
sorted_df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,over_under,seven_day_accuracy
0,2021-10-21,Bogdan,ATL,G,DAL,vs.,15.5,-115,16.57,1.07,15.5,11.0,o,0.47
1,2021-10-23,Bogdan,ATL,G,CLE,@,14.5,-115,15.29,0.79,14.5,5.0,o,0.45
2,2021-10-25,Bogdan,ATL,G,DET,vs.,15.5,-115,12.85,-2.65,15.5,14.0,u,0.45
3,2021-10-27,Bogdan,ATL,G,NOR,@,12.5,-125,13.70,1.20,12.5,12.0,o,0.45
4,2021-10-28,Bogdan,ATL,G,WAS,@,12.5,-105,16.72,4.22,12.5,14.0,o,0.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14477,2022-03-08,Ziaire Williams,MEM,G,NOR,vs.,6.5,-110,7.30,0.80,6.5,16.0,o,0.46
14478,2022-03-11,Ziaire Williams,MEM,G,NYK,vs.,7.5,-115,7.30,-0.20,7.5,13.0,u,0.50
14479,2022-03-15,Ziaire Williams,MEM,G,IND,@,8.5,-130,7.60,-0.90,8.5,11.0,u,0.46
14480,2022-03-23,Ziaire Williams,MEM,G,BKN,vs.,7.5,-105,6.20,-1.30,7.5,0.0,u,0.46


In [61]:
# Determine if website pick was a win or loss (for fun, potential spinoff angle).
conditions = [
    ((sorted_df['Result'] < sorted_df['Pick']) & (sorted_df['over_under'] == 'u')),
    ((sorted_df['Result'] < sorted_df['Pick']) & (sorted_df['over_under'] == 'o')),
    ((sorted_df['Result'] > sorted_df['Pick']) & (sorted_df['over_under'] =='o')),
    ((sorted_df['Result'] > sorted_df['Pick']) & (sorted_df['over_under'] == 'u'))
    ]

# create a list of the values we want to assign for each condition
values = ['1', '0', '1', '0']

# create a new column and use np.select to assign values to it using our lists as arguments
sorted_df['successful_pick'] = np.select(conditions, values)

In [62]:
sorted_df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,over_under,seven_day_accuracy,successful_pick
0,2021-10-21,Bogdan,ATL,G,DAL,vs.,15.5,-115,16.57,1.07,15.5,11.0,o,0.47,0
1,2021-10-23,Bogdan,ATL,G,CLE,@,14.5,-115,15.29,0.79,14.5,5.0,o,0.45,0
2,2021-10-25,Bogdan,ATL,G,DET,vs.,15.5,-115,12.85,-2.65,15.5,14.0,u,0.45,1
3,2021-10-27,Bogdan,ATL,G,NOR,@,12.5,-125,13.70,1.20,12.5,12.0,o,0.45,0
4,2021-10-28,Bogdan,ATL,G,WAS,@,12.5,-105,16.72,4.22,12.5,14.0,o,0.42,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14477,2022-03-08,Ziaire Williams,MEM,G,NOR,vs.,6.5,-110,7.30,0.80,6.5,16.0,o,0.46,1
14478,2022-03-11,Ziaire Williams,MEM,G,NYK,vs.,7.5,-115,7.30,-0.20,7.5,13.0,u,0.50,0
14479,2022-03-15,Ziaire Williams,MEM,G,IND,@,8.5,-130,7.60,-0.90,8.5,11.0,u,0.46,0
14480,2022-03-23,Ziaire Williams,MEM,G,BKN,vs.,7.5,-105,6.20,-1.30,7.5,0.0,u,0.46,1


In [85]:
sorted_df.to_csv('betting_data.csv', index=False)